# CBA Policy Portal Sraper
CBA make downloading the policy difficult by
1. Having a SSO based login that makes request-http automation difficult
2. Using ASPX to dynamically load content.

Oover come this using by loggign in with the browser, then ctting and pasting the auth cookies into this notebook.

Scraper Code
1. Only searchs for links in div#wrapper
2. Has code to deal with ASPX/DynamicContent
3. Limited to commbroker domain

In [1]:
from  scraper import WebScraper
from loguru import logger
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import os

class WBC_SGB(WebScraper):
    def __init__(self, domain:str, cookie:str, download_path:str, search_only:bool=False):
        super().__init__(domain, cookie, download_path, class_names = ["div#wrapper"])
        self.search_only = search_only

    def scrape(self, url:str, depth:int, starting_page:bool = True, ) -> None:

        logger.info(f"Scraping {depth}: {url}")
        # Skip if we have already visited the page, or if we have exceeded the depth
        # or if the url is not in the domain, or if the url is not an html/ASPX file
        if self.skip(url, depth):
            return
        
        path = urlparse(url).path
        extension = os.path.splitext(path)[-1].lower()
     
        # If aspx find the base html then extract the links and recurse without saving content
        if extension == ".aspx":
            html_urls = self.aspx_link_to_html_link(url)
            for html_url in html_urls:
                self.scrape(html_url, depth=depth, starting_page=starting_page)
            return
        
        # download all resources except starting page
        if starting_page:
            html_content = self.fetch_content(url)
        else:
            if self.search_only:
                logger.info(f"Saving {url}")           
                html_content = self.fetch_content(url)
            else:
                html_content = self.save_url(url)
        
        # We have an HTML file and  have not yet exceeded the depth 
        # find all the links in wrapper and recurse
        assert( extension == ".html")
        soup = BeautifulSoup(html_content)
        for link in soup.select('div#wrapper a'):
            if href := link.get('href'):
                if not urlparse(href).netloc:
                    href = urljoin(self.domain, href)

                self.scrape(href, depth=depth-1, starting_page=False)
        return

# Download Comm Broker

In [3]:
import shutil

directory_path=r"/home/kevinmcisaac/Projects/policy-pal-pages/policy/CBA/CommBroker/"

if os.path.exists(directory_path):
    shutil.rmtree(directory_path)  
os.makedirs(directory_path) 

#Need depth 3 to get to 
wbc = WBC_SGB("https://www.commbroker.com.au", 
              cookie="cbpopup=50; s_cc=true; s_nr=1735871648417; s_sq=cfsadviser-cb-prd%3D%2526pid%253Dcb%25253Apopup%25253Aclosebutton%2526pidt%253D1%2526oid%253Dhttps%25253A//www.commbroker.com.au/Net/Documentum/credit-policy/acceptable-excluded-purposes.aspx%2526ot%253DA; ASP.NET_SessionId=qgk3typnu0vhtcdl5o5bjbot; cbid=6KuDmFHhuI4j94IkWTMErzTEp5GLolQz4oTPujB_--VY17RZnjKb3_KZBV1ZtC7b60E_tN4wE4oqm0oNB_BF3Umc7dXBwEAWaHkMhRk-oLhq6pux_ApP5oBVt5o12_ZRVDJZenzNONq_Rim8K9hq30fJsKTEp39XDtXxYcPcV8ymtDy4Y6v5ebIKwDlo_KlkRNN0fOoVZ91XtJpAYHoY6iO7XzGr9cWDBE_StwN4UiRlRITTNRntjE1WCLz0o6DN2ONjJWZxm6dJq8WxDefDSiEQgoOIfjY7Mhalg5qegSwfsHaxrBiRSf_xxeErjHlYOK7feROR_DBZ1JXGXIgrSwKGj_FKO9R0hxfSbkdI973-xc0mpS9WMSbMM99-GAjSm8BgHG3RdaS8myt6bZScdbss8iC87oUlAPtXG8IM01dnt-A2diGS_cw0Cl22dxAzE1kwZbbP2C5GjfJ7WOVkpgpc9tvIDTQiCL4YMi37NkV-6CfYisCTSckLa1kOxkwVmZAEoDG_DGfpqY_kzLOiisYId5VglPYjaVFjB7sttXMnwMi61mk4gHLQcFInDNzaLIOhHbsAzIXdWTEgDQGvG4g7tXr7uFq7KIQtbQVxUJBeOoO44RMDzR32RrZN9JiHD7PYTb9qgtAMdh8BXw3UEg4mAsmDzRlemDAxNHL9EFN1BXjs9qPsOa0rBUd-uOX51xjbwv4taeXQmd67QdJ3iaAUGKvlxw59-JImcEQ2lQ518rmL5TvjpQouKX9sdTEwmzfW7nOQA5AVzP4aRJ-yeMz0po8aDAVdTNPueV1094jqZZzdG5fly5-TLI_6nNbOYeQCmjPtSSLrhA85LqBo8AhjtoEi3FEZdV0zI1aZueP5EJ6xMKO15iPnH5OgwW0-6xJjAPZT8_dNxJ6NFTCP9cW1104u7XEEy7xJqY1iJzHLqUqmpks1GM4v0W9g7qhlxmMzvcjtXJMVHopalgI5WVS1-WaKBmMOjrpS5dfrIXSJRJJK6lHiLn27XrO1k2tq9G92D3eSYynl5rQ27M9UcbdSnxvk7-uvKuJrB5Fv_jOh1rXDABPoeYcdZHNbrlg2n_iEfIBPSKDDWojkhDPjaaLkI6C3VcnAbwEKtOeC876bCsQTzOz4n1q6SQLEhsf_wtBsBMFZ-Trgj1IJN2AbEWKN7wLgLRZ1msIiOe3A7u-f3bnEBXCarerIQpSn5udVwz53BjRXrxDychFzgwV4c5ozTX4dbI7Ptg6hV6KZI7npM3EjOyaWleiUQ9Ex1chSGKZ6ejiLJQDnmd4rFdbcvF5y3Jjmo9aQ7ZPIkJVzTu5JySev-vO4-tVnt7AgcNwqjSY43ge9ERqSS6FWPvGioiEQ0QsTKPP0XTokD0h-LdTI46o30FL3NF69_529rfazqfw0ec7njoAqA-50VWbkPc3rOEdXIEccCHNetU2gFW00R6gZ3G3hxy0OIGT4q5jOuhFMK_ghH-jItUj78QtdhHnbj1sAC9ylZ74cf6Ch8t9VKGaMByB6qhhsHyw5AAsDheAsa2gvRVXS4EtHH9xe5xs0JzbsVZ46S9gXocW0YOxI3Afbin2AmlkgG3NRZQTsaRYsEGwsLUTtz6pFkqFJl8Bco5zNe0wran7MaX0eDMn6MDuN5zgHFZxQey6ATTWqUsLYXPGQ9F_MLv8VMThPNEFa57egYJDqTSJg-q2LndyOIibazjEB3f4RaILi4FSbobY3fgK3mP0_Er1OG-sveStjM7kZ6hak5hzP99cWWNx5cN5xoFSlGdwv6v6OEiM9lQnD0UQr3bLRHJHA7DDFTrY6Bd_n1An8TAPy6uAjfMsBkpN74UWWy1bGaEm6bAZeSZnHGRlpLb4eualD86RlA8V0oc6t9xa_Iliow1vGVR8pHia9Af1clnt75LOJbizxATmwsFILlTe1WQe_XM6XbYitpRLuhTUluthoVHa3vP64SZe9VI3RPoA_29EutW-5uGG3EpYD_AUo4ycEth-m1EXVIyBHkk6QSMipsm5KBe6mTpyWC67HagMiK6K0NCT25jwEpOen844fKkSAXFcRyJ5GBnukAAbEPlRPdG-pBMbjQyASxR79L-1oFskNzy9-Uh4LOO8LIxz9Z2i9SXD7sylGfQpiHPZ9gRsY9uLGgQ-FxzaKs-zJDBDAO5Dsf-o5x2Hq0UE6qa4zP95vkiW9tUBUkSezITcj1WoHPCyVMyVpwq9j4T6XEsA48MVeEuzEBB2aaInTeZ4Hv4jkbZ0aWIkIwfwHIxGAgHbDfF077JIVfVyXW3pHxZZQ_OHCkIqHAevOWQsAin13MetCi--jhOVWuXFrgqy8uXY4-O4NhTcdIR5PVO6STnr4Kt3MQl2u2Dv3MrEpn4XeWlynSqsNIwGF2yoYh9CrC-RUsxop3zXByo_MV8SLvi9Fscud9KjY191ZzbJhdLVzwrkZrIS8zrxW-JLDwKCZwQOqOzdu2yyki_MgjY4mGLzVA4voYutsh8Tgb6yhRQtllUhEXUWTkreUv2auCeormny0fvWbWhTVh99XLaZlhsCNAl1ANCMJR0xnHf7JAkqEW2VD0hF0KmYzARv5Tx1HXf_O_cJHor-6n3rJ_L9P0A_ifn9fKFJjn8coMXri4dnfgtvbdO_3BO7FKDgfBga9Q17Pd40TkfEliA05BwidrClU6n7UNXo_r9RbZE1CJ_k-KsvXzyXYh6RTVwNlq51lKf1QCIZhRmbFdvCGOVL188rDjEG57k1MYCQOI2U_H0nsLY88OltfF6FO7KPB1u7kGIaIX3z4r4nJUt_ZCXpwooQI6v0iDPkOYchvobHGydtOIcIwR1AdsusAcSMe0ZlinzCgnOgyaEwDyPi3R3yRS3kdWwLqKoXwHKk5ewVZC3Yp1lXLlGAGDSIp5fNHbHFPmlj0tBxUv5aDDRIHdIlAexA5MpGMgjEJzcLd7aY4mYpRko4DBUezpWAKWl2ZhlJV5tC8PaEgX-EtGkm4BeXz1dv6nme6F7eGj0i2-6anNk8hChw2CY7brZ_4X_p0x7oHSk0-PaieacOLP_nr5rYiMjhapy_ip9ZWHHmQ9ThyC6SnodOWRu1ZiMaxxzWbBQRUn5VgfUDFZuxZN1LJNhPjfehhd5CIvemz8EU4Fx6cfovyzjG5T86_SDeAqcudsFdkxZd_bea7Fax_dArz_gV1f56dpX54pqNVtsWw3uwizfZIJ8YjS964WRMHt8VFuOVBd1hz8nPtmTetS3UcsfjxGtnkc0-tQ6TKWfYSFCn6bYACHkDBNsjuATJyag9LE2PkW4CMiBHIkMjbnVnCavxeDdav7wQfwu8nm-dLvcQh_gJfEmPKlIVAkKqkp9RScO2czWyXom7lFmK_UdvYnMK1029YkTXRh4FcFfCHf0ExWxj268z1Yqgx73uoPiKgLVbzHliCyL0I0kRjQqdO5xP8hamp_sZVms5heJ7cbw; htsm=exp=1736037398~acl=/*~hmac=2b026263a2616ec30e1bec4ef392740e5e028e09996a41f1bdb32228f6c5a803; bm_mi=A256223D84C28EC8841E9AD40D156B2C~YAAQRQNiaHsbnjGUAQAA7gHJMxrfBMvnn0M0cCzkhVkNMHQ7QGzZnd4fjSif/XOaa5cCwR9w4SfChaNqoT9fCFWdkKBswNO1H7Gpg2ipxCSYAUK76c41WkZOJ7RsiiphXZg+k3nnB7Lp4J0EYiYB3SeINC21/KxTXShk7/kQ98aCxRR7SuXvzRpaRtJIGI+ZYEbxhmh/Uttz9RGceF9Jl12dKfUSjtTACxq/GtTDZVPMKCqGA3kMQKiRmW8Z5BP9VB1Xx8zIMA4IkAZohi4LPUAjG4vw+A1FwzTSe+zyh3snvwDFCB1WBTdhYYi10WX9I6oWsRIL1B8cosAA1ZeRdHw2vEYyqHn7i7IAhLrDlmB2hcVgGe/GP4K/6S5aXA==~1; bm_sz=C897468F265E1D72BE5FCF5F854FA190~YAAQRQNiaH0bnjGUAQAA7gHJMxp2KgReR9RaxwwDV43VfTP3+cobZ8Kx/2HNAoPsGZs0OMVMY9MGiw0fN6FnVvTH4EfNl7isIJQnYvEBLFQGjApudRauF/ElFdOwt3m4WchnJ7WSseUwHzn+/CenKTmfrQN+cK1Z91+cAv6upH2Gkm9HKDuJ+DHd+0+5kmWh3E20OxdVL6z1FB2bL1VoKEaGOOKrkdSHZhgBYxTcj/k++X3v7bIaeoVYMYLj9WH5ZIFnsZRq5tyH7QI/GuJMTdrr2pLaC20ExJAp+1HgQ/MtDgfb64XXTyFhprItXFtPQFDExRsti7r2Ol0jjWsL6OPXT5YBK7nHks4h/DjYICJrE7fHZ7l1IAWby/5RGJ8u3IF00CGHRkstvQLyAmgecGsL6Nzohj6uyHz/mScdoA==~4338743~3490374; ak_bmsc=6275F89EF5A98CE5A99378FEFF9D078D~000000000000000000000000000000~YAAQXgNiaBJ0UyqUAQAACwXJMxoqxaxrjTYvL/EZqJQNK3wfq/sPS536hV1jFkin1foXaOYGDyxLi81HQRmhg8R6JB/HJIAX7OkLgTD4eEQOpzyXu+Uvf6XK8a2VYjuDXt3ri3Hv8CyU7Ie5Kmq6gjb8NrM/lK3U1+TZmNciFBMuTs8nKVOhsRVd8j6xE0SA66Y05xomDv0MhyPntdizXtApgW9rOdPsTw+tpAv9xoVBqFBgMm9praxR7thv+lJY2l1OVAud11emKmsEku+ktLfa2wKlmEOyhYCbEQu/RUTeC8KwD0+9mvVgsF3/ELaGwKt+fI1VhfwVz8hXkpN/zTVR0XS7zxPyHX/1xIpe4yC1pzJ90DNejTCgxpo9Ginx08RopmEwHnr3d5KuPCXsWs6LsxJYWOqVIpIv/JVDDtkf79SxufpXclFRNgk3/8XbexwZuMs+kcoxy9rCL3Qdg7PmXW/lFv04xlugHGColE0tL354HV9+vjsH/TPUXdhizLBs66IWnzFMoAScAIByKVOKRlzWM0I=; _abck=CDA8A3A4C475B653B887DAA636D183BF~-1~YAAQXgNiaKOEUyqUAQAAIabJMw08rNK7ZuP9m+bA6QvuGvq8vNrUgbGyN2fJc56U0zRBVNKIj0ARtwhBQM/4QbSyqjYYox6AQWhXWlkK9rMPS3qAmP1TU064ylTmBRmeoYmZ2ZF4fyy6zpgxy0RCFDeXt244DineRpdHoE6uWDucNCYNl/Rc3M8nnjf9GnCrXm1CC08GTB0Yf3yXPErT4eKgLpWuTVlzgu8TRNnkVt1c/YC+UI0oX5aCuYQofauA2GvQcctKZTiCdPiFVXdaCfu+l5UA9sK8MDUK53hgUWyYwEsbjb9VfsC6p6fwf60uXz3m9JqTKd8tSqPdkhpg3Dc+Do+11fXxS/+QvgeZljnoKnD2TLjmB0EB0D5ATFFhjuFZfEIg/0RPbfHdYb0L8lEpCFNKkDw6kzkMgRfuSJk2HIxEss5V2RQyhWM7cGI39PNeSwVnqwrIoHyyBNl7IqzLVL19etmZgg1Nzij7kuuqKv74bIH+iXmtP5yMOH7JHFkReNHt3EaAn/0Bzvk=~-1~-1~-1; bm_sv=B568E041C5BD113905F69F9596BD83C5~YAAQXgNiaCaFUyqUAQAASKzJMxpD+imSEdvCrX+PziYbShdfzMSoxv1FyC1N9TeO76lioKQsrajg26vkbbpphUeRMKacexLqPEOoD2l7sL8H1xCVvLnKp1suVPoKwR9XCFEAOIVYy52VO2YGmo5Nb6ADCaaevft2GG9FwyxzNKHXG4fJAQGUk8V3FbTUcE3ItFxLONMnggvMN5Ir/WI8mUUlWR4u9Ld9xLgYmpO+rWOzYZdGPyaX/UdA7vvMOKEuzgDtP+bbpg==~1", 
              download_path= directory_path, 
              search_only=False)
wbc.scrape("https://www.commbroker.com.au/Net/Documentum/credit-policy/default.aspx", depth=16)
wbc.scrape("https://www.commbroker.com.au/Net/Documentum/processes/default.aspx", depth=16)

print(len(wbc.download))

2025-01-05 08:22:27.000 | INFO     | __main__:scrape:14 - Scraping 16: https://www.commbroker.com.au/Net/Documentum/credit-policy/default.aspx


2025-01-05 08:22:27.510 | INFO     | __main__:scrape:14 - Scraping 16: https://www.commbroker.com.au/content/commbroker/en/private/credit-policy/default.html
2025-01-05 08:22:27.703 | INFO     | __main__:scrape:14 - Scraping 15: https://ecomms.cba.com.au/cv/4e28c09365e1c179bd89ab131200ec829530c314
2025-01-05 08:22:27.706 | INFO     | scraper:skip:103 - Skipping domain https://ecomms.cba.com.au/cv/4e28c09365e1c179bd89ab131200ec829530c314
2025-01-05 08:22:27.707 | INFO     | __main__:scrape:14 - Scraping 15: https://www.commbroker.com.au/Net/Documentum/credit-policy/acceptable-excluded-purposes.aspx
2025-01-05 08:22:27.924 | INFO     | __main__:scrape:14 - Scraping 15: https://www.commbroker.com.au/content/commbroker/en/private/credit-policy/acceptable-excluded-purposes.html
2025-01-05 08:22:28.075 | INFO     | scraper:save_url:49 - Downloading https://www.commbroker.com.au/content/commbroker/en/private/credit-policy/acceptable-excluded-purposes.html
2025-01-05 08:22:28.167 | INFO     | 

216
